In [1]:
import re
from nltk.tokenize import TweetTokenizer
import pandas as pd

import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\I059953\Desktop\Hate speech\Hate speech\labeled_data.csv") 

In [3]:
all_sentences = []
tweet_tokenizer = TweetTokenizer()

for i in range(df.shape[0]):
  tweet = ""
  s = df.iloc[i]['tweet']
  # lowercase
  s = s.lower()
  # remove urls
  s = re.sub(r'https?:\/\/\S+', ' ', s)
  s = re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', s)
  # change @... and #abc to thisisatag
  s = re.sub(r'\s([@#][\w_-]+)', ' thisisatag ', s)
  # remove non-letter chars
  s = re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', s)
  # thisisatag -> TAG
  s = re.sub(r" thisisatag ", ' TAG ', s)
  # remove rt
  s = re.sub(r" rt ", ' ', s)
  # remove unecessary spacing
  s = s.strip()
  s = tweet_tokenizer.tokenize(s)
  # remove punctuation after tokenization
  for j in s:
    if j not in string.punctuation:
      tweet = tweet + j + " "
  all_sentences.append(tweet[:-1]) 

In [4]:
clean_text_df = pd.DataFrame(columns=['tweet', 'label'])
for i in range(len(all_sentences)):
  # add tweet and label to clean_df
  tweet = all_sentences[i]
  label = df.iloc[i]['class']
  # is hate speech
  if label == 0:
    label = 1
  # not hate speech 
  else:
    label = 0
  clean_text_df.loc[i] = [tweet, label]

In [5]:
myTokenizer = Tokenizer()
myTokenizer.fit_on_texts(all_sentences)
token_sequences = myTokenizer.texts_to_sequences(all_sentences)
padded_sequences = pad_sequences(token_sequences)

In [6]:
clean_df = pd.DataFrame(columns=['tweet', 'label'])
for i in range(len(padded_sequences)):
  # add tweet and label to clean_df
  clean_df.loc[i] = [padded_sequences[i].tolist(), df.iloc[i]['class']]